In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import qutip
from scipy.linalg import expm
from scipy.constants import hbar

np.set_printoptions(precision=2)

# 2 qubits with exchange interaction

Define the two-spin Pauli matrices, e.g.:

$\sigma_{1,z} = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix} \otimes \mathcal{I}$

$\sigma_{1,x} = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix} \otimes \mathcal{I}$

$\sigma_{1,y} = \begin{pmatrix} 0 & -i \\ i & 0 \end{pmatrix} \otimes \mathcal{I}$

In [ ]:
identity = np.array([[1,0],[0,1]])

sigma1Z = np.kron(np.array([[1,0],[0,-1]]),identity)
sigma1X = np.kron(np.array([[0,1],[1,0]]),identity)
sigma1Y = np.kron(np.array([[0,-1j],[1j,0]]),identity)

sigma2Z = np.kron(identity,np.array([[1,0],[0,-1]]))
sigma2X = np.kron(identity,np.array([[0,1],[1,0]]))
sigma2Y = np.kron(identity,np.array([[0,-1j],[1j,0]]))

Define the Hamiltonian $\mathcal{H} = \frac{J}{2} \mathbf(S_1 S_2) = J \frac{\hbar}{4} (\sigma_{x,1}\sigma_{x,2} + \sigma_{y,1}\sigma_{y,2} + \sigma_{z,1}\sigma_{z,2})$

In [ ]:
J = 2 * np.pi

H = J * hbar/4 * (sigma1X@sigma2X + sigma1Y@sigma2Y + sigma1Z@sigma2Z)
print(H)

Initialise qubit states and form a two qubit state $| \psi \rangle = | \psi_1 \rangle \otimes | \psi_2 \rangle$

In [ ]:
psi1 = np.array([1,0])
psi2 = np.array([1,1])/np.sqrt(2)
psi0 = np.kron(psi1,psi2)
print(psi0)

Evolve the state in time using $| \psi_t \rangle = U(t) | \psi_0 \rangle$ with $U(t) = e^{\frac{-i H t}{\hbar}}$

In [ ]:
time = np.linspace(0,0.5,101)
psiT = []

for t in time:
    psiT.append(expm(-1j*H*t/hbar) @ psi0)

Save the expectation values of x, y and z (e.g. $\langle x_1 \rangle = \langle \psi_t | \sigma_{1,x} | \psi_t \rangle$)

In [ ]:
x1 = [(p.conj().T @ sigma1X @ p).real for p in psiT]
y1 = [(p.conj().T @ sigma1Y @ p).real for p in psiT]
z1 = [(p.conj().T @ sigma1Z @ p).real for p in psiT]

x2 = [(p.conj().T @ sigma2X @ p).real for p in psiT]
y2 = [(p.conj().T @ sigma2Y @ p).real for p in psiT]
z2 = [(p.conj().T @ sigma2Z @ p).real for p in psiT]

Plot the evolution of the states

In [ ]:
def qubitPlots(time,x,y,z):
    bsPlot = qutip.Bloch()
    for xi,yi,zi in zip(x,y,z):
        bsPlot.add_points([xi, yi, zi])
    bsPlot.render(bsPlot.fig, bsPlot.axes)
    bsPlot.fig.set_size_inches([25,5])

In [ ]:
qubitPlots(time,[x1,x2],[y1,y2],[z1,z2])